In [1]:
import rqdatac as rq
rq.init()

/Users/mistletoe/miniconda3/envs/examples/lib/python3.11/site-packages/rqdatac/client.py:245: UserWarning: Your account will be expired after  329 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [21]:
symbols = ['A', 'AG', 'AL', 'AP', 'AU', 'B', 'BU', 'C', 'CF', 'CS', 'CU', 'CY',
       'FB', 'FG', 'HC', 'J', 'JD', 'L', 'M', 'MA', 'NI', 'OI', 'PB', 'PP',
       'RB', 'RM', 'RU', 'SF', 'SM', 'SN', 'SR', 'TA', 'V', 'Y', 'ZC', 'ZN']
dfs = [] 
for sym in symbols:
    df = rq.futures.get_member_rank(sym,start_date='20180101',end_date='20241231')
    df = df.reset_index()
    dfs.append(df)

rank_all = pd.concat(dfs, ignore_index=True)
rank_all

,trading_date,commodity_id,rank,member_name,volume,volume_change
0,2018-01-02,A,1,海通期货,8462,-512
1,2018-01-02,A,2,国投安信,5264,-3467
2,2018-01-02,A,3,中信期货,4050,-4818
3,2018-01-02,A,4,西部期货,3741,511
4,2018-01-02,A,5,招商期货,3469,-3072
...,...,...,...,...,...,...
1177652,2024-12-31,ZN,16,华闻期货,6046,-6014
1177653,2024-12-31,ZN,17,浙商期货,5991,1038
1177654,2024-12-31,ZN,18,东方财富,4324,-897
1177655,2024-12-31,ZN,19,宝城期货,4285,-3211


In [6]:
import pandas as pd
rank_all = pd.concat(dfs, ignore_index=True)
rank_all

,trading_date,commodity_id,rank,member_name,volume,volume_change
0,2018-01-02,A,1,国投安信,14469,-2145
1,2018-01-02,A,2,中信期货,14338,-55
2,2018-01-02,A,3,中粮期货,12621,-31
3,2018-01-02,A,4,国泰君安,12228,304
4,2018-01-02,A,5,华泰期货,6753,-188
...,...,...,...,...,...,...
1177730,2024-12-31,ZN,16,东吴期货,4951,-54
1177731,2024-12-31,ZN,17,云晨期货,4853,193
1177732,2024-12-31,ZN,18,广发期货,4665,-581
1177733,2024-12-31,ZN,19,华鑫期货,4496,-49


In [7]:
rank_all.to_parquet('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/institutional_trading.parquet')

In [30]:
# 1 机构投资者交易量
ITV = (
    rank_all
      .groupby(['trading_date', 'commodity_id'], as_index=False)      
      [['volume', 'volume_change']].sum()                            
      .rename(columns={'trading_date':'date',
                       'commodity_id':'underlying_symbol',
                       'volume': 'IT',
                       'volume_change': 'ITC'})       
)

ITV
ITV = ITV.merge(com_1d_bar, 
                on=['underlying_symbol', 'date'], 
                how = 'inner')
ITV


,date,underlying_symbol,IT,ITC,volume,open_interest,close
0,2018-01-02,A,56692,-32756,92242.0,233118.0,2839.0
1,2018-01-02,AG,85370,-57245,137022.0,602744.0,4790.0
2,2018-01-02,AL,363965,-18990,250552.0,296254.0,13000.0
3,2018-01-02,AP,10344,-4342,16426.0,12360.0,5857.0
4,2018-01-02,AU,34437,-9619,54760.0,255972.0,327.9
...,...,...,...,...,...,...,...
57691,2024-12-31,SR,465088,-160665,245954.0,367499.0,5961.0
57692,2024-12-31,TA,945773,194192,557474.0,1146290.0,4892.0
57693,2024-12-31,V,1449198,409502,919633.0,897647.0,5290.0
57694,2024-12-31,Y,654384,107083,460515.0,667652.0,7712.0


In [82]:
com_1d_bar = pd.read_parquet('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/com_1d_bar.parquet', engine = 'fastparquet').reset_index()
com_1d_bar = com_1d_bar[['underlying_symbol', 'date', 'volume', 
                        'open_interest', 'close']]
com_1d_bar['date'] = pd.to_datetime(com_1d_bar['date'])
com_1d_bar = com_1d_bar[com_1d_bar['date'] > '2018-01-01']
com_1d_bar

,underlying_symbol,date,volume,open_interest,close
1944,A,2018-01-02,92242.0,233118.0,2839.0
1945,A,2018-01-03,134428.0,233774.0,2859.0
1946,A,2018-01-04,89870.0,237044.0,2869.0
1947,A,2018-01-05,167556.0,235366.0,2872.0
1948,A,2018-01-08,94634.0,233268.0,2888.0
...,...,...,...,...,...
140351,ZN,2024-12-25,139070.0,142766.0,25335.0
140352,ZN,2024-12-26,140056.0,150232.0,25305.0
140353,ZN,2024-12-27,155198.0,143526.0,25085.0
140354,ZN,2024-12-30,142279.0,139968.0,25235.0


In [22]:
rank_all_trade = rank_all
rank_all_trade.to_parquet('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/institutional_trading_tradevolume.parquet')

In [32]:
# 2 机构投资者持卖仓量
ISV = (rank_all_short.groupby(['trading_date', 'commodity_id'], as_index=False)      
    [['volume', 'volume_change']].sum()                            
    .rename(columns={'trading_date':'date',
                    'commodity_id':'underlying_symbol',
                    'volume': 'IS',
                    'volume_change': 'ISC'})       
)
ITV = ITV.merge(ISV, 
                on=['underlying_symbol', 'date'], 
                how = 'inner')

ITV

,date,underlying_symbol,IT,ITC,volume,open_interest,close,IS,ISC
0,2018-01-02,A,56692,-32756,92242.0,233118.0,2839.0,101747,-920
1,2018-01-02,AG,85370,-57245,137022.0,602744.0,4790.0,179670,2418
2,2018-01-02,AL,363965,-18990,250552.0,296254.0,13000.0,292010,915
3,2018-01-02,AP,10344,-4342,16426.0,12360.0,5857.0,5438,-278
4,2018-01-02,AU,34437,-9619,54760.0,255972.0,327.9,34476,567
...,...,...,...,...,...,...,...,...,...
57691,2024-12-31,SR,465088,-160665,245954.0,367499.0,5961.0,362036,-3506
57692,2024-12-31,TA,945773,194192,557474.0,1146290.0,4892.0,1025341,-10227
57693,2024-12-31,V,1449198,409502,919633.0,897647.0,5290.0,811555,38633
57694,2024-12-31,Y,654384,107083,460515.0,667652.0,7712.0,633741,-14231


In [33]:
# 3 机构投资者持买仓量 rank_all_long
IBV = (rank_all_long.groupby(['trading_date', 'commodity_id'], as_index=False)      
    [['volume', 'volume_change']].sum()                            
    .rename(columns={'trading_date':'date',
                    'commodity_id':'underlying_symbol',
                    'volume': 'IB',
                    'volume_change': 'IBC'})       
)
ITV = ITV.merge(IBV, 
                on=['underlying_symbol', 'date'], 
                how = 'inner')

ITV


,date,underlying_symbol,IT,ITC,volume,open_interest,close,IS,ISC,IB,IBC
0,2018-01-02,A,56692,-32756,92242.0,233118.0,2839.0,101747,-920,71025,-763
1,2018-01-02,AG,85370,-57245,137022.0,602744.0,4790.0,179670,2418,157795,884
2,2018-01-02,AL,363965,-18990,250552.0,296254.0,13000.0,292010,915,260611,1811
3,2018-01-02,AP,10344,-4342,16426.0,12360.0,5857.0,5438,-278,5199,-311
4,2018-01-02,AU,34437,-9619,54760.0,255972.0,327.9,34476,567,82472,2249
...,...,...,...,...,...,...,...,...,...,...,...
57691,2024-12-31,SR,465088,-160665,245954.0,367499.0,5961.0,362036,-3506,387593,-3739
57692,2024-12-31,TA,945773,194192,557474.0,1146290.0,4892.0,1025341,-10227,901293,-8966
57693,2024-12-31,V,1449198,409502,919633.0,897647.0,5290.0,811555,38633,663177,28808
57694,2024-12-31,Y,654384,107083,460515.0,667652.0,7712.0,633741,-14231,542824,-7700


In [34]:
ITV['IOI'] = (ITV['IB'] - ITV['IS']) / (ITV['IB'] + ITV['IS'])
ITV

,date,underlying_symbol,IT,ITC,volume,open_interest,close,IS,ISC,IB,IBC,IOI
0,2018-01-02,A,56692,-32756,92242.0,233118.0,2839.0,101747,-920,71025,-763,-0.177818
1,2018-01-02,AG,85370,-57245,137022.0,602744.0,4790.0,179670,2418,157795,884,-0.064822
2,2018-01-02,AL,363965,-18990,250552.0,296254.0,13000.0,292010,915,260611,1811,-0.056818
3,2018-01-02,AP,10344,-4342,16426.0,12360.0,5857.0,5438,-278,5199,-311,-0.022469
4,2018-01-02,AU,34437,-9619,54760.0,255972.0,327.9,34476,567,82472,2249,0.410405
...,...,...,...,...,...,...,...,...,...,...,...,...
57691,2024-12-31,SR,465088,-160665,245954.0,367499.0,5961.0,362036,-3506,387593,-3739,0.034093
57692,2024-12-31,TA,945773,194192,557474.0,1146290.0,4892.0,1025341,-10227,901293,-8966,-0.064386
57693,2024-12-31,V,1449198,409502,919633.0,897647.0,5290.0,811555,38633,663177,28808,-0.100614
57694,2024-12-31,Y,654384,107083,460515.0,667652.0,7712.0,633741,-14231,542824,-7700,-0.077273


In [85]:
ITV.to_parquet('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/ITV.parquet')

In [1]:
import pandas as pd
las = pd.read_csv('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/all_rolling_predictions_lasso_1min.csv')
codes = ['A', 'AG', 'AL', 'AP', 'AU', 'B', 'BU', 'C', 'CF', 'CS', 'CU', 'CY',
       'FB', 'FG', 'HC', 'J', 'JD', 'L', 'M', 'MA', 'NI', 'OI', 'PB', 'PP',
       'RB', 'RM', 'RU', 'SF', 'SM', 'SN', 'SR', 'TA', 'V', 'Y', 'ZC', 'ZN']

las['code'] = (
    las.groupby('datetime').cumcount()          
      .map(dict(enumerate(codes)))
)
las

,datetime,y_true,y_pred,code
0,2018-04-02 09:03:00,-0.001013,-2.433967e-05,A
1,2018-04-02 09:04:00,-0.001691,7.235570e-05,A
2,2018-04-02 09:05:00,0.002029,1.208710e-04,A
3,2018-04-02 09:06:00,-0.001014,-1.453990e-04,A
4,2018-04-02 09:07:00,-0.001354,7.240473e-05,A
...,...,...,...,...
12732403,2024-12-30 14:56:00,0.000000,-3.916357e-06,ZN
12732404,2024-12-30 14:57:00,0.000198,-2.473360e-06,ZN
12732405,2024-12-30 14:58:00,0.000000,-1.586909e-06,ZN
12732406,2024-12-30 14:59:00,-0.000198,-3.602595e-06,ZN


In [3]:
import numpy as np
df = las.copy()
df['datetime'] = pd.to_datetime(df['datetime'])
df['trade_date'] = df['datetime'].dt.normalize()


def oos_r2(group: pd.DataFrame) -> float:
    y_true = group['y_true']
    y_pred = group['y_pred']
    sse = np.square(y_true - y_pred).sum()        
    sst = np.square(y_true).sum()    
    return 1 - sse / sst if sst != 0 else np.nan     

# daily_r2 = (
#     df.groupby(['code', 'trade_date'])
#       .apply(oos_r2)
#       .reset_index(name='r2_oos')
#       .sort_values(['trade_date', 'code'])
# )

code_r2 = (
    df.groupby(['code'])
               .apply(oos_r2)
               .reset_index(name = 'r2_oos')
               .sort_values(['code']))

code_r2

/var/folders/wd/dqz448js18390ywvw7fmx0800000gn/T/ipykernel_86089/2665993273.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(oos_r2)


,code,r2_oos
0,A,0.005510
1,AG,0.002450
2,AL,0.009660
3,AP,0.002702
4,AU,0.002899
5,B,0.010687
6,BU,0.019251
7,C,0.054640
8,CF,0.009931
9,CS,0.022868


In [40]:
daily_r2.columns = ['underlying_symbol', 'date', 'r2_oos']
ITV = ITV.merge(daily_r2, 
                on = ['underlying_symbol', 'date'], 
                how = 'inner')
ITV

,date,underlying_symbol,IT,ITC,volume,open_interest,close,IS,ISC,IB,IBC,IOI,r2_oos
0,2018-04-02,A,143900,-33798,161998.0,126438.0,2963.00,89774,8240,61853,6412,-0.184143,0.009563
1,2018-04-02,AG,104164,-36278,192436.0,770888.0,4519.00,240910,-3098,237270,-5022,-0.007612,0.067895
2,2018-04-02,AL,343564,55964,311646.0,292588.0,11660.00,264061,1098,220088,6139,-0.090825,0.010728
3,2018-04-02,AP,431646,64531,635948.0,73706.0,5175.00,31691,5487,33703,5956,0.030767,-0.002231
4,2018-04-02,AU,47880,-4321,73540.0,315638.0,320.25,29068,-1779,86904,-1738,0.498707,0.080735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53858,2024-12-30,SR,654605,164714,346797.0,369702.0,5969.00,365549,4188,391456,3584,0.034223,0.000282
53859,2024-12-30,TA,806704,-131021,456936.0,1145066.0,4856.00,1038301,-13596,913572,-216,-0.063902,0.017782
53860,2024-12-30,V,1051301,64523,684891.0,864574.0,5326.00,781073,-58206,635495,-32959,-0.102768,-0.007173
53861,2024-12-30,Y,556242,-18934,377586.0,685903.0,7700.00,650868,4937,553110,7238,-0.081196,0.009719


#### FMB predictive regression

In [ ]:
import statsmodels.api as sm

df = ITV.copy()
df['date'] = pd.to_datetime(df['date'])
factor_cols = ['IT', 'ITC', 'IOI', 'RT', 'RTC']
# for col in factor_cols:
#     df[f'{col}_lag1'] = (
#         df.sort_values('date')
#           .groupby('underlying_symbol')[col]
#           .shift(1)
#     )

# lag_cols = [f'{c}_lag1' for c in factor_cols]

lag_cols = factor_cols
def zscore(x):
    return (x - x.mean()) / x.std(ddof=0)

cols_to_z = ['r2_oos'] + lag_cols

df_z = (
    df.dropna(subset=cols_to_z)          
      .groupby('date', group_keys=False)
      .apply(lambda g: g.assign(**{c + '_z': zscore(g[c]) for c in cols_to_z}))
)

y_col  = 'r2_oos_z'
x_cols = [c + '_z' for c in lag_cols]

records = []

for trade_date, g in df_z.groupby('date'):
    if len(g) < len(x_cols) + 2:    
        continue

    X = sm.add_constant(g[x_cols])
    y = g[y_col]
    res = sm.OLS(y, X).fit()

    rec = {'date': trade_date, 'alpha': res.params['const']}
    rec.update({f'beta_{i+1}': res.params[col] for i, col in enumerate(x_cols)})
    records.append(rec)

betas = pd.DataFrame(records).sort_values('date').reset_index(drop=True)
betas

/var/folders/wd/dqz448js18390ywvw7fmx0800000gn/T/ipykernel_8718/2780528538.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(**{c + '_z': zscore(g[c]) for c in cols_to_z}))


,date,alpha,beta_1,beta_2,beta_3,beta_4,beta_5
0,2018-04-03,4.857226e-17,0.189753,-0.134822,-0.107402,-0.497349,0.288529
1,2018-04-04,-1.734723e-17,-0.098636,0.034555,-0.063933,0.069976,-0.369939
2,2018-04-09,-6.245005e-17,-1.098296,0.121159,-0.026378,0.681211,-0.034123
3,2018-04-10,2.775558e-17,-1.137412,0.374537,-0.056093,0.919518,-0.462103
4,2018-04-11,-5.551115e-17,0.025357,0.199927,-0.165880,-0.135357,-0.115736
...,...,...,...,...,...,...,...
1580,2024-12-24,-3.642919e-17,-0.463794,-0.445497,-0.172029,-0.874025,-0.747628
1581,2024-12-25,-3.339343e-17,-0.770269,-0.261806,-0.221062,-1.227691,-0.009611
1582,2024-12-26,-6.245005e-17,-0.950593,-0.032885,-0.197146,-1.262587,0.092299
1583,2024-12-27,-4.163336e-17,-0.582217,0.612433,-0.313377,-0.807420,0.587139


In [66]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

nw_lag = 20
rows = []

for col in betas.columns.drop('date'):
    s = betas[col].dropna()
    T = len(s)
    if T < nw_lag + 10:
        continue

    res = sm.OLS(s, np.ones(T)).fit(
        cov_type='HAC', cov_kwds={'maxlags': nw_lag}
    )
    rows.append({
        'coef'  : col.replace('beta_','').upper(),
        'mean'  : res.params[0],
        'NW_t'  : res.tvalues[0],
        'N_day' : T
    })

df = pd.DataFrame(rows)

def stars(t):
    at = abs(t)
    if at > 2.58:
        return '***'
    elif at > 1.96:
        return '**'
    elif at > 1.65:
        return '*'
    else:
        return ''

df['sig'] = df['NW_t'].apply(stars)

# 排版：保留 4 位小数 / 2 位 t 值
daily_summary = (
    df.assign(mean  = df['mean'].round(4),
              NW_t  = df['NW_t'].round(2))
      .set_index('coef')
      .loc[:, ['mean', 'NW_t', 'sig', 'N_day']]
      .rename(columns={'mean':'mean_coef',
                       'NW_t':'NW_tstat'})
)

print(daily_summary)

       mean_coef  NW_tstat  sig  N_day
coef                                  
ALPHA    -0.0000     -0.33        1585
1        -0.0844     -2.31   **   1585
2        -0.0765     -5.05  ***   1585
3        -0.0953     -6.36  ***   1585
4        -0.1705     -4.74  ***   1585
5         0.0129      0.76        1585


#### Panel regression

In [75]:
from linearmodels.panel import PanelOLS
import pandas as pd

df = ITV.copy()
dfp = df.set_index(['underlying_symbol', 'date']).sort_index()

for col in ['IT','ITC','IOI','RT','RTC','r2_oos']:
    dfp[col + '_z'] = (
        dfp.groupby('date')[col]
           .transform(lambda x: (x - x.mean()) / x.std(ddof=0))
    )

y    = dfp['r2_oos_z']
X    = sm.add_constant(dfp[['RT_z','RTC_z']])

mod  = PanelOLS(
    y, X,
    entity_effects=True,   # γ_i
    time_effects=True      # δ_t
)

res = mod.fit(cov_type='clustered',
              cluster_entity=True,
              cluster_time=True)
print(res.summary)

/Users/mistletoe/miniconda3/envs/examples/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:               r2_oos_z   R-squared:                        0.0027
Estimator:                   PanelOLS   R-squared (Between):             -0.0191
No. Observations:               53805   R-squared (Within):               0.0027
Date:                Mon, Jul 14 2025   R-squared (Overall):             -0.0030
Time:                        18:30:19   Log-likelihood                -6.803e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      71.288
Entities:                          34   P-value                           0.0000
Avg Obs:                       1582.5   Distribution:                 F(2,52185)
Min Obs:                       1534.0                                           
Max Obs:                       1585.0   F-statistic (robust):             9.6741
                            

/Users/mistletoe/miniconda3/envs/examples/lib/python3.11/site-packages/linearmodels/panel/results.py:89: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


In [84]:
ITV = ITV.merge(df, 
                on = ['underlying_symbol', 'date'], 
                how = 'inner')
ITV

,date,underlying_symbol,IT,ITC,volume_x,open_interest_x,close_x,IS,ISC,IB,IBC,IOI,r2_oos,RT,RTC,volume_y,open_interest_y,close_y,ILLIQ
0,2018-04-02,A,143900,-33798,161998.0,126438.0,2963.00,89774,8240,61853,6412,-0.184143,0.009563,18098.0,NaN,161998.0,126438.0,2963.00,9.405476e-12
1,2018-04-02,AG,104164,-36278,192436.0,770888.0,4519.00,240910,-3098,237270,-5022,-0.007612,0.067895,88272.0,NaN,192436.0,770888.0,4519.00,3.887317e-12
2,2018-04-02,AL,343564,55964,311646.0,292588.0,11660.00,264061,1098,220088,6139,-0.090825,0.010728,-31918.0,NaN,311646.0,292588.0,11660.00,3.085501e-12
3,2018-04-02,AP,431646,64531,635948.0,73706.0,5175.00,31691,5487,33703,5956,0.030767,-0.002231,204302.0,NaN,635948.0,73706.0,5175.00,5.864319e-12
4,2018-04-02,AU,47880,-4321,73540.0,315638.0,320.25,29068,-1779,86904,-1738,0.498707,0.080735,25660.0,NaN,73540.0,315638.0,320.25,7.609648e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53858,2024-12-30,SR,654605,164714,346797.0,369702.0,5969.00,365549,4188,391456,3584,0.034223,0.000282,-307808.0,-92549.0,346797.0,369702.0,5969.00,2.295766e-12
53859,2024-12-30,TA,806704,-131021,456936.0,1145066.0,4856.00,1038301,-13596,913572,-216,-0.063902,0.017782,-349768.0,16855.0,456936.0,1145066.0,4856.00,2.752203e-12
53860,2024-12-30,V,1051301,64523,684891.0,864574.0,5326.00,781073,-58206,635495,-32959,-0.102768,-0.007173,-366410.0,-31995.0,684891.0,864574.0,5326.00,1.506750e-12
53861,2024-12-30,Y,556242,-18934,377586.0,685903.0,7700.00,650868,4937,553110,7238,-0.081196,0.009719,-178656.0,22800.0,377586.0,685903.0,7700.00,1.368611e-12


In [83]:
import pandas as pd
import numpy as np

df = com_1d_bar.copy()
df = df.sort_values(['underlying_symbol', 'date']).copy()
df['ret'] = (
    df.groupby('underlying_symbol')['close']
      .pct_change()
)

df['dollar_vol'] = df['close'] * df['volume']
df['illiq_daily'] = df['ret'].abs() / df['dollar_vol']

# 4) 5 日滚动均值 —— 每只合约单独滚动
df['ILLIQ'] = (
    df.groupby('underlying_symbol')['illiq_daily']
      .transform(lambda x: x.rolling(window=5, min_periods=5).mean())
)

df = df.drop(columns=['ret', 'dollar_vol', 'illiq_daily'])
df

,underlying_symbol,date,volume,open_interest,close,ILLIQ
1944,A,2018-01-02,92242.0,233118.0,2839.0,NaN
1945,A,2018-01-03,134428.0,233774.0,2859.0,NaN
1946,A,2018-01-04,89870.0,237044.0,2869.0,NaN
1947,A,2018-01-05,167556.0,235366.0,2872.0,NaN
1948,A,2018-01-08,94634.0,233268.0,2888.0,NaN
...,...,...,...,...,...,...
140351,ZN,2024-12-25,139070.0,142766.0,25335.0,1.166685e-12
140352,ZN,2024-12-26,140056.0,150232.0,25305.0,1.159099e-12
140353,ZN,2024-12-27,155198.0,143526.0,25085.0,1.525013e-12
140354,ZN,2024-12-30,142279.0,139968.0,25235.0,1.728822e-12


In [32]:
from linearmodels.panel import PanelOLS
import pandas as pd
import statsmodels.api as sm

df = ITV.copy()
dfp = df.set_index(['underlying_symbol', 'date']).sort_index()

for col in ['volume_y','open_interest_y','ILLIQ','IT','ITC','IOI','RT','RTC','LD', 'r2_oos']:
    dfp[col + '_z'] = (
        dfp.groupby('date')[col]
           .transform(lambda x: (x - x.mean()) / x.std(ddof=0))
    )

y    = dfp['r2_oos_z']
X    = sm.add_constant(dfp[['RT_z','RTC_z','IT_z','ITC_z','IOI_z','LD_z','volume_y_z','open_interest_y_z','ILLIQ_z']])

mod  = PanelOLS(
    y, X,
    entity_effects=True,   # γ_i
    time_effects=True      # δ_t
)

res = mod.fit(cov_type='clustered',
              cluster_entity=True,
              cluster_time=True)
print(res.summary)

/Users/mistletoe/miniconda3/envs/examples/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


ValueError: exog does not have full column rank. If you wish to proceed with model estimation irrespective of the numerical accuracy of coefficient estimates, you can set check_rank=False.

### price limits

In [7]:
import pandas as pd
com_1d_bar = pd.read_parquet('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/com_1d_bar.parquet', engine = 'fastparquet').reset_index()
# com_1d_bar = com_1d_bar[['underlying_symbol', 'date', 'volume', 
#                         'open_interest', 'close']]
com_1d_bar['date'] = pd.to_datetime(com_1d_bar['date'])
com_1d_bar = com_1d_bar[com_1d_bar['date'] > '2018-01-01']
com_1d_bar = com_1d_bar.iloc[:, 2:]
com_1d_bar

,underlying_symbol,date,prev_settlement,day_session_open,high,low,settlement,volume,prev_close,open,open_interest,limit_up,limit_down,total_turnover,close,dominant_id
1944,A,2018-01-02,2823.0,3590.0,2868.0,2783.0,2835.0,92242.0,2826.0,2783.0,233118.0,3004.0,2642.0,0,2839.0,A1805
1945,A,2018-01-03,2835.0,3636.0,2875.0,2831.0,2852.0,134428.0,2839.0,2839.0,233774.0,3016.0,2654.0,0,2859.0,A1805
1946,A,2018-01-04,2852.0,3658.0,2870.0,2842.0,2856.0,89870.0,2859.0,2866.0,237044.0,3034.0,2670.0,0,2869.0,A1805
1947,A,2018-01-05,2856.0,3684.0,2900.0,2858.0,2876.0,167556.0,2869.0,2870.0,235366.0,3038.0,2674.0,0,2872.0,A1805
1948,A,2018-01-08,2876.0,3676.0,2890.0,2852.0,2873.0,94634.0,2872.0,2877.0,233268.0,3059.0,2693.0,0,2888.0,A1805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140351,ZN,2024-12-25,25020.0,25355.0,25380.0,25185.0,25285.0,139070.0,25140.0,25250.0,142766.0,26775.0,23260.0,0,25335.0,ZN2502
140352,ZN,2024-12-26,25285.0,25500.0,25550.0,25235.0,25390.0,140056.0,25335.0,25390.0,150232.0,27060.0,23505.0,0,25305.0,ZN2502
140353,ZN,2024-12-27,25390.0,25420.0,25430.0,25050.0,25235.0,155198.0,25305.0,25335.0,143526.0,27170.0,23605.0,0,25085.0,ZN2502
140354,ZN,2024-12-30,25235.0,25310.0,25320.0,25055.0,25205.0,142279.0,25085.0,25080.0,139968.0,27005.0,23460.0,0,25235.0,ZN2502


In [10]:
tick = 0.1
df = com_1d_bar.copy()
df['hit_up']   = (df['high']  >= df['limit_up']   - tick) # 触及涨停
df['close_up'] = (df['close'] >= df['limit_up']   - tick) # 封涨停板

df['hit_down']   = (df['low']   <= df['limit_down'] + tick) # 封跌停板
df['close_down'] = (df['close'] <= df['limit_down'] + tick) # 触及封停
df

,underlying_symbol,date,prev_settlement,day_session_open,high,low,settlement,volume,prev_close,open,open_interest,limit_up,limit_down,total_turnover,close,dominant_id,hit_up,close_up,hit_down,close_down
1944,A,2018-01-02,2823.0,3590.0,2868.0,2783.0,2835.0,92242.0,2826.0,2783.0,233118.0,3004.0,2642.0,0,2839.0,A1805,False,False,False,False
1945,A,2018-01-03,2835.0,3636.0,2875.0,2831.0,2852.0,134428.0,2839.0,2839.0,233774.0,3016.0,2654.0,0,2859.0,A1805,False,False,False,False
1946,A,2018-01-04,2852.0,3658.0,2870.0,2842.0,2856.0,89870.0,2859.0,2866.0,237044.0,3034.0,2670.0,0,2869.0,A1805,False,False,False,False
1947,A,2018-01-05,2856.0,3684.0,2900.0,2858.0,2876.0,167556.0,2869.0,2870.0,235366.0,3038.0,2674.0,0,2872.0,A1805,False,False,False,False
1948,A,2018-01-08,2876.0,3676.0,2890.0,2852.0,2873.0,94634.0,2872.0,2877.0,233268.0,3059.0,2693.0,0,2888.0,A1805,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140351,ZN,2024-12-25,25020.0,25355.0,25380.0,25185.0,25285.0,139070.0,25140.0,25250.0,142766.0,26775.0,23260.0,0,25335.0,ZN2502,False,False,False,False
140352,ZN,2024-12-26,25285.0,25500.0,25550.0,25235.0,25390.0,140056.0,25335.0,25390.0,150232.0,27060.0,23505.0,0,25305.0,ZN2502,False,False,False,False
140353,ZN,2024-12-27,25390.0,25420.0,25430.0,25050.0,25235.0,155198.0,25305.0,25335.0,143526.0,27170.0,23605.0,0,25085.0,ZN2502,False,False,False,False
140354,ZN,2024-12-30,25235.0,25310.0,25320.0,25055.0,25205.0,142279.0,25085.0,25080.0,139968.0,27005.0,23460.0,0,25235.0,ZN2502,False,False,False,False


In [22]:
limit_dates = (df[['hit_up','close_up','hit_down','close_down']]
                 .any(axis=1))                # 行 bool
limit_dates = df.loc[limit_dates, 'date'].unique()  # ndarray

In [25]:
ITV = pd.read_parquet('/Volumes/T7Shield/PhD/RA/Sparse Signals in the Cross-Section of  Returns/output/ITV.parquet')
ITV['LD'] = ITV['date'].isin(limit_dates).astype(bool)
ITV['LD'] = ITV['LD'].astype(int)

In [26]:
ITV

,date,underlying_symbol,IT,ITC,volume_x,open_interest_x,close_x,IS,ISC,IB,IBC,IOI,r2_oos,RT,RTC,volume_y,open_interest_y,close_y,ILLIQ,LD
0,2018-04-02,A,143900,-33798,161998.0,126438.0,2963.00,89774,8240,61853,6412,-0.184143,0.009563,18098.0,NaN,161998.0,126438.0,2963.00,9.405476e-12,0
1,2018-04-02,AG,104164,-36278,192436.0,770888.0,4519.00,240910,-3098,237270,-5022,-0.007612,0.067895,88272.0,NaN,192436.0,770888.0,4519.00,3.887317e-12,0
2,2018-04-02,AL,343564,55964,311646.0,292588.0,11660.00,264061,1098,220088,6139,-0.090825,0.010728,-31918.0,NaN,311646.0,292588.0,11660.00,3.085501e-12,0
3,2018-04-02,AP,431646,64531,635948.0,73706.0,5175.00,31691,5487,33703,5956,0.030767,-0.002231,204302.0,NaN,635948.0,73706.0,5175.00,5.864319e-12,0
4,2018-04-02,AU,47880,-4321,73540.0,315638.0,320.25,29068,-1779,86904,-1738,0.498707,0.080735,25660.0,NaN,73540.0,315638.0,320.25,7.609648e-11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53858,2024-12-30,SR,654605,164714,346797.0,369702.0,5969.00,365549,4188,391456,3584,0.034223,0.000282,-307808.0,-92549.0,346797.0,369702.0,5969.00,2.295766e-12,0
53859,2024-12-30,TA,806704,-131021,456936.0,1145066.0,4856.00,1038301,-13596,913572,-216,-0.063902,0.017782,-349768.0,16855.0,456936.0,1145066.0,4856.00,2.752203e-12,0
53860,2024-12-30,V,1051301,64523,684891.0,864574.0,5326.00,781073,-58206,635495,-32959,-0.102768,-0.007173,-366410.0,-31995.0,684891.0,864574.0,5326.00,1.506750e-12,0
53861,2024-12-30,Y,556242,-18934,377586.0,685903.0,7700.00,650868,4937,553110,7238,-0.081196,0.009719,-178656.0,22800.0,377586.0,685903.0,7700.00,1.368611e-12,0
